Part 1: Build the Friendship Network

• Represent students as nodes in an undirected graph.
• Friendships are edges with random but realistic generation logic:

– Higher probability of friendship within same groups/classes

– Some students are more popular (higher degree)

– Friends-of-friends become more likely (clustering)

• Assign weights (1–10) to represent friendship strength

– Lower weight = closer connection

 Use an adjacency list with weights

In [ ]:
import numpy as np
from itertools import combinations
import random
num_people = 1000
class_size = 50
threshold = 0.85 #probablity above which they are friends iteratively i have updated from 0.8,0.9,0.98,0.99 to finally chose which could be optimal
people = np.arange(num_people)
classes = people // class_size

pairs = np.array(list(combinations(people, 2)))# Generate all unique unordered pairs
class1 = classes[pairs[:, 0]]# Identify same class or different class
class2 = classes[pairs[:, 1]]
same_class = class1 == class2

# now let us assign probablities using random function
probs = np.where(
    same_class,
    np.random.normal(0.73, 0.1, size=len(pairs)),
    np.random.normal(0.2, 0.1, size=len(pairs))
)

# friend or not can be determined and stored in this array
friend_flags = (probs > threshold).astype(int)

# Create final_data array: [person1, person2, probability, is_friend]
final_data = np.hstack([
    pairs,
    probs.reshape(-1, 1),
    friend_flags.reshape(-1, 1)
])

from collections import defaultdict

# let us build friend map from initial friendships
friend_map = defaultdict(set)
for p1, p2, _, flag in final_data:
    if flag == 1:
        friend_map[int(p1)].add(int(p2))
        friend_map[int(p2)].add(int(p1))

# now we can create quick pair index
pair_index = { (min(int(p1), int(p2)), max(int(p1), int(p2))): idx for idx, (p1, p2, *_ ) in enumerate(final_data) }

# Apply friend-of-friend probability boost our idea is that if they are friend of friend they will have additional probablitty than initial probablity
for a, neighbors in friend_map.items():
    for b, c in combinations(neighbors, 2):
        b, c = sorted((b, c))
        idx = pair_index.get((b, c))
        if idx is not None and final_data[idx, 3] == 0:
            final_data[idx, 2] += 0.1
            final_data[idx, 2] = min(final_data[idx, 2], 1.0)
            if final_data[idx, 2] > threshold:
                final_data[idx, 3] = 1


adjacency_list = [[] for _ in range(num_people)]# weighted adjacency list can be created now

# we can assign weights inversely proportional to probability (1 to 10 scale)
for p1, p2, prob, flag in final_data:
    if flag == 1:
        weight = int(round(10 * (1 - prob)))  # Closer friends have lower weights
        adjacency_list[int(p1)].append((int(p2), weight))
        adjacency_list[int(p2)].append((int(p1), weight))

Part 2: Analyze Friend Groups

• Use DFS or BFS to count connected components

• Report:

– Number of friend groups

– Size of smallest and largest group

In [ ]:
# let us use DFS using iterative method to find connected components
def neighbours(a, adj):
    return [neigh for neigh, _ in adj[a]]

def preorder(start, adj, marked):
    order = []
    stack = [start]
    while stack:
        a = stack.pop()
        if not marked[a]:
            marked[a] = 1
            order.append(a)
            for w in neighbours(a, adj):
                if not marked[w]:
                    stack.append(w)
    return order

group_no = np.zeros(num_people, dtype=int)
group_current = 1

for i in range(num_people):
    if group_no[i] != 0:
        continue
    marked = [0] * num_people
    members = preorder(i, adjacency_list, marked)
    for m in members:
        group_no[m] = group_current
    group_current += 1
total_groups = group_current - 1
group_sizes = np.bincount(group_no.astype(int))[1:]

print("Total number of friendship groups:", total_groups)
print("Smallest group size:", group_sizes.min())
print("Largest group size:", group_sizes.max())


Total number of friendship groups: 25
Smallest group size: 1
Largest group size: 50


Part 3: Find Closest Friendship Paths

• Select 5 random student pairs

• Use Dijkstra’s algorithm to find shortest “friendship effort” paths

• For 1 pair, also apply A* using a basic heuristic:

– e.g., same class = heuristic 0, different = 5

• Report paths and compare with Dijkstra


In [ ]:
# first let us write the code for dijikistra and A* then we can implement the needed ones
def dijkstra(start, end, adj):
    dist = [float('inf')] * len(adj)
    prev = [None] * len(adj)
    visited = [False] * len(adj)
    dist[start] = 0

    nodes = list(range(len(adj)))

    while nodes:
        # Find the node with the smallest distance
        u = min((n for n in nodes if not visited[n]), key=lambda x: dist[x], default=None)
        if u is None or u == end:
            break

        nodes.remove(u)
        visited[u] = True

        for v, weight in adj[u]:
            if not visited[v]:
                alt = dist[u] + weight
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u

    # Reconstruct path
    path = []
    u = end
    while u is not None:
        path.insert(0, u)
        u = prev[u]
    return path, dist[end]

def heuristic(u, v):
    return 0 if (u // 50 == v // 50) else 5

def a_star(start, end, adj):
    open_set = [start]
    came_from = [None] * len(adj)
    g_score = [float('inf')] * len(adj)
    f_score = [float('inf')] * len(adj)

    g_score[start] = 0
    f_score[start] = heuristic(start, end)

    while open_set:
        # Pick node with lowest f_score
        current = min(open_set, key=lambda x: f_score[x])
        if current == end:
            break

        open_set.remove(current)

        for neighbor, weight in adj[current]:
            tentative_g = g_score[current] + weight
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, end)
                if neighbor not in open_set:
                    open_set.append(neighbor)

    # Reconstruct path
    path = []
    current = end
    while current is not None:
        path.insert(0, current)
        current = came_from[current]
    return path, g_score[end]

# we can randomly select 5 students and then test
random.seed(42)
pairs_to_test = [(random.randint(0, 999), random.randint(0, 999)) for _ in range(5)]
print("Random student pairs:", pairs_to_test)
for s, t in pairs_to_test:
    path, cost = dijkstra(s, t, adjacency_list)
    print(f"Dijkstra: {s} -> {t} | Cost: {cost} | Path: {path}")
s, t = pairs_to_test[0]
path, cost = a_star(s, t, adjacency_list)
print(f"A*: {s} -> {t} | Cost: {cost} | Path: {path}")

Random student pairs: [(654, 114), (25, 759), (281, 250), (228, 142), (754, 104)]
Dijkstra: 654 -> 114 | Cost: inf | Path: [114]
Dijkstra: 25 -> 759 | Cost: inf | Path: [759]
Dijkstra: 281 -> 250 | Cost: inf | Path: [250]
Dijkstra: 228 -> 142 | Cost: inf | Path: [142]
Dijkstra: 754 -> 104 | Cost: inf | Path: [104]
A*: 654 -> 114 | Cost: inf | Path: [114]
